In [1]:
import ddsp
import ddsp.training


In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

%cd ..
!ls
from core import *

/Users/trippgordon/Desktop/Code/sonic/NoisyGAN
Data        README.md   core        models      submit.sh
Notebooks   __init__.py depricated  notes_ideas train.py


INFO:absl:tf versionn: 2.1.0


In [3]:
# get a dataset 
output_tfrecord_path = './Data/piano/piano30s.tfrecord'
dataset = DDSP_DATASET(output_tfrecord_path)

# play a sample
play(dataset.getAudio())

dataset.getAudio().shape

TensorShape([64000])

In [4]:
#  start: 'audio', 'f0_confidence', 'f0_hz', 'loudness_db'
singleSample = dataset.getSample()
singleSample.keys()

dict_keys(['audio', 'f0_confidence', 'f0_hz', 'loudness_db'])

In [5]:
# add: 'f0_scaled', 'ld_scaled'
preprocessor = ddsp.training.preprocessing.DefaultPreprocessor(time_steps=1000)
preprocessed = preprocessor(singleSample, training=True)
preprocessed.keys() 

dict_keys(['audio', 'f0_confidence', 'f0_hz', 'loudness_db', 'f0_scaled', 'ld_scaled'])

In [6]:
#add: 'amps', 'harmonic_distribution', 'noise_magnitudes'
decoder = ddsp.training.decoders.RnnFcDecoder(
            rnn_channels = 256,
            rnn_type = 'gru',
            ch = 256,
            layers_per_stack = 1,
            output_splits = (('amps', 1),
                             ('harmonic_distribution', 45),
                             ('noise_magnitudes', 45)))

decoded = decoder(preprocessed)
decoded.keys()

dict_keys(['audio', 'f0_confidence', 'f0_hz', 'loudness_db', 'f0_scaled', 'ld_scaled', 'amps', 'harmonic_distribution', 'noise_magnitudes'])

In [7]:
# add:
additive = ddsp.synths.Additive(n_samples=DEFAULT_N_SAMPLES,
                                sample_rate=DEFAULT_SAMPLE_RATE, # this is defined in my_ddsp_utils
                                name='additive')
noise = ddsp.synths.FilteredNoise(window_size=0,
                                  initial_bias=-10.0,
                                  name='noise')
add = ddsp.processors.Add(name='add')
reverb = ddsp.effects.Reverb(name='reverb', trainable=True)
dag = [(additive, ['amps', 'harmonic_distribution', 'f0_scaled']),
       (noise, ['noise_magnitudes']),
       (add, ['noise/signal', 'additive/signal']),
       (reverb, ['add/signal'])]
processor_group = ddsp.processors.ProcessorGroup(dag=dag,
                                                 name='processor_group')

audio_gen = processor_group(decoded)
audio_gen.numpy().shape

(1, 64000)

In [8]:
singleSample['audio'].numpy()[0]

-0.030972093

In [9]:
audio_gen = audio_gen.numpy().squeeze()
play(audio_gen)